In [1]:
# DEEPFAKE IMAGE DETECTION MODEL

In [2]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'deepfake-detection-challenge:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F16880%2F858837%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240315%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240315T052208Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D484dad2ac8ce8e2ca4c4cca1c836da3b4b837ae7b6a7fe5bb323299b4f1b6e1f1d5f304ccda7d9e3944f22ef1c22d77c0148d0ffbe1c91b2bb58dbb9fe72ba3890c1ff20b6bd2110220ae024d9e85af3aec57406c6f7e9b4546bf0ba584a2bd1b23ca8c1ce31ec7514198ac3de2f8090e96b675ea180027c4a9c3bfb265ee828d999d1a5cca3e2b0d2d1b8a6de3a31ad9f20a66a36296db6cb6553528c29e973e82b64282c2d7da57dff438509ba262c3985c5796040bd40077a6be04cfa912af1b2a27d8e96e13b6b5b39a2eafaa89ec7ebe739f633410f5ed629e37e9c6112af8b944d5bda3ac3232f70b9724f7e7ceccf332988bd28b516f290a77be120b8,haarcascades:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F13405%2F18147%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240315%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240315T052209Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D667128cb8d97924fc657b1ec5db26fc99dfee047069a1dae3a1cbfb826ccf214527c995427f76a75834db290f2056818a1c70e4ded89516bf6089d361dd82d107077061d1667ac4f4329d409aafa7d9836f1cb55bc8b69ab476d5fa155615a14233050e895da0bc3648380ef99b705b405c45940ff26e6778486b31087b85c9d84421139e0f758ba11a338e9ee1ff9b164c23ca4a4741026db4587488531e11ab827654b289e39cd3f6bca3ce5d857aa782edda3ae6c4ee39b34cc371269abf6d63141da91f2aae36a64b2460d8afd057dd1237f8c47c92c87431de3d6a7e6df44a4d50a608ed962871817e948f3991f93c41b74ed52277faaca7d5f9f6721db,ffmpeg-static-build:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F444558%2F842050%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240315%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240315T052209Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0691d18b4a30b9412972acd12cf5d12353fe3fd17b1f5023e27c134956d7cf2ad8b01f999b9b6861ba20e24282533de153c92e25d66c9b9868466a7db32260aa8bbfaae2a39386b06342c0d7ab4cc5aa0d12f9ae718e137d7ace730f1a7fc8d6d736cf24bdf3360784f1a9756cfb6991715100360c2c6e12f4368b231a5a805549fb8d51773101c2e0a78516e42162554d83507ac4f36b5aa7223da30df38a5fe73ef47235e5c11dc19f95298a3cda62855956ebc153ab83114e89471f1096545cf6ec1355b327ae1c26bb7b98c87f3a7b5374884706630b9c380b9ad2bcc94b79fddb62e58cbf4158e803582c783459c6e2c6308fa5d67b8ad0d6bb5d00e044,deepfake:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F451078%2F893807%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240315%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240315T052209Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6dc28182d10de532684cb6a92340dffe34b2b68468a1b64e8e76e41700acab523b7271acec9ab3f0928c850606b2c451751c0f81ecabef378bf9b98470d28e3a6a461e3e69df9fec0dfe786a0e7b9e3201218ab6937fb4401533b9ccc3807b6a74d467689d513a69aef1296ede892112f044672deea98aaa10ad0847198e1938712aa7b82416e0b52feb5e887240734f2e7ade222a73f356c88cd69350358ce8d8ffba1d72e3c7c9d67a82fb7829c572cb9f1667809ebd4b894d9c2028f2eec807093f3e95a905435dd1ff75010fad787827e684ba39d56d6f1e69435e4309d37244733d8e9cbc979f66d237b329f8b01c38b271f177ba32ddd8d2aa92dd9b0e,deep-voice-deepfake-voice-recognition:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3579787%2F6358196%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240315%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240315T052209Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7d8c5a45307d8f8add9fdf488cfe9a30be41cd967753754b5f7409d561a6c1646902e34b85b9698e0fd1ad1cc258ebf1cb86c0346b74a82d85f7c81896824bec581aecadef439b123698d48188e90a4cd8a77c53f6a98c44123a47ca461c0bff16eab1a41fc239b7104766f6470a0a80b8942d733759566bc643f7e8640f2b8ed557675b9894d4b06b4a1d2fe44674d91c8960263b6effe18e89aabeaae1c7094bad54c4d7aa9236940a611cdef81331d07a28a37d969c62e6d81951d4b51828cefd28675bc1cd4c5f3a12acbf3f2730f84edd17bee0af326cac060c8eb3bec13ea301443593ba2252b3ca7ac263f56e7b4ba5e310eb3c1278d1b280e4f586c6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 4439352219 bytes downloaded
Downloaded and uncompressed: deepfake-detection-challenge
[==================================================] 1519369 bytes downloaded
Downloaded and uncompressed: haarcascades
[==================================================] 37817225 bytes downloaded
Downloaded and uncompressed: ffmpeg-static-build
[==================================================] 800415004 bytes downloaded
Downloaded and uncompressed: deepfake
[==================================================] 3956840339 bytes downloaded
Downloaded and uncompressed: deep-voice-deepfake-voice-recognition
Data source import complete.


In [4]:
!pip install -U --upgrade tensorflow

In [5]:
import sys
import sklearn
import tensorflow as tf

import cv2
import pandas as pd
import numpy as np

import plotly.graph_objs as go
from plotly.offline import iplot
from matplotlib import pyplot as plt

In [6]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [7]:
tf.__version__

'2.16.1'

In [8]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [9]:
import os

def get_data():
    return pd.read_csv('../input/deepfake-faces/metadata.csv')

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
meta=get_data()
meta.head()

FileNotFoundError: [Errno 2] No such file or directory: '../input/deepfake-faces/metadata.csv'